In [1]:
import numpy as np
import tensorflow as tf
import statistics
import matplotlib.pyplot as plt
from tensorflow.keras import layers,models,optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

### Question 1

Since we have a binary classification problem, what is the best loss function for us?

- `binary crossentropy`

- `focal loss`

- `mean squared error`

- `categorical crossentropy`

Note: since we specify an activation for the output layer, we don't need to set `from_logits=True`

In [2]:
TRAIN_DIR = "data/train"
TEST_DIR = "data/test"
datagen = ImageDataGenerator()
train_generator = datagen.flow_from_directory(TRAIN_DIR, class_mode="binary",target_size=(150,150))
validation_generator = datagen.flow_from_directory(TEST_DIR, class_mode="binary",target_size=(150,150))

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


### Model
For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

- The shape for input should be `(150, 150, 3`)

- Next, create a convolutional layer (`Conv2D`):
    
    - Use 32 filters
    
    - Kernel size should be `(3, 3)` (that's the size of the filter)
    
    - Use `'relu'` as activation
                                    
- Reduce the size of the feature map with max pooling (`MaxPooling2D`)
    - Set the pooling size to `(2, 2)`
- Turn the multi-dimensional result into vectors using a `Flatten` layer
- Next, add a `Dense` layer with 64 neurons and `'relu'` activation
- Finally, create the `Dense` layer with 1 neuron - this will be the output
    - The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use SGD with the following parameters:

- `SGD(lr=0.002, momentum=0.8)`

In [3]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(units=64,activation='relu'))
model.add(layers.Dense(units=1,activation='sigmoid'))
model.compile(optimizer=optimizers.SGD(learning_rate=0.002, momentum=0.8),loss="binary_crossentropy",metrics=["accuracy"])

In [4]:
#model.fit(train_generator)

__ANSWER__: a) `binary crossentropy`

### Question 2

What's the total number of parameters of the model? You can use the `summary` method for that.

- 9215873
- 11215873
- 14215873
- 19215873

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

__ANSWER__: b) `11215873`

### Generators and Training

For the next two questions, use the following data generator for both train and validation:

```python
ImageDataGenerator(rescale=1./255)
```

- We don't need to do any additional pre-processing for the images.

- When reading the data from train/val directories, check the class_mode parameter. Which value should it be for a binary classification problem?

- Use `batch_size=20`

- Use `shuffle=True` for both training and validation

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)
```

In [6]:
datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_generator = datagen.flow_from_directory(TRAIN_DIR, class_mode="binary",shuffle=True,target_size=(150,150))
validation_generator = datagen.flow_from_directory(TEST_DIR, class_mode="binary",shuffle=True,target_size=(150,150))

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [ ]:
history=model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
50/50 [==============================] - 22s 438ms/step - loss: 0.6320 - accuracy: 0.6267 - val_loss: 0.5552 - val_accuracy: 0.6726
Epoch 2/10
50/50 [==============================] - 11s 229ms/step - loss: 0.4739 - accuracy: 0.7955 - val_loss: 0.4592 - val_accuracy: 0.7944
Epoch 3/10
50/50 [==============================] - 12s 240ms/step - loss: 0.4086 - accuracy: 0.8331 - val_loss: 0.3804 - val_accuracy: 0.8503
Epoch 4/10
50/50 [==============================] - 12s 232ms/step - loss: 0.3607 - accuracy: 0.8576 - val_loss: 0.3628 - val_accuracy: 0.8528
Epoch 5/10
50/50 [==============================] - 14s 271ms/step - loss: 0.3258 - accuracy: 0.8701 - val_loss: 0.3287 - val_accuracy: 0.8655
Epoch 6/10
50/50 [==============================] - 12s 239ms/step - loss: 0.2996 - accuracy: 0.8846 - val_loss: 0.3196 - val_accuracy: 0.8858
Epoch 7/10
 2/50 [>.............................] - ETA: 11s - loss: 0.2686 - accuracy: 0.9062

In [ ]:
history.history.keys()

### Question 3

What is the median of training accuracy for all the epochs for this model?

- 0.40
- 0.60
- 0.90
- 0.20

In [ ]:
statistics.median(history.history["accuracy"])

__ANSWER__: c) `0.90`

### Question 4

What is the standard deviation of training loss for all the epochs for this model?

- 0.11
- 0.66
- 0.99
- 0.33

In [ ]:
statistics.stdev(history.history["loss"])

__ANSWER__: a) `0.11`

### Data Augmentation

For the next two questions, we'll generate more data using data augmentations.


Add the following augmentations to your training data generator:

- `rotation_range=40,`


- `width_shift_range=0.2,`


- `height_shift_range=0.2,`


- `shear_range=0.2,`


- `zoom_range=0.2,`


- `horizontal_flip=True,`


- `fill_mode='nearest'`

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

train_generator = datagen.flow_from_directory(TRAIN_DIR, class_mode="binary",target_size=(150,150))
validation_generator = datagen.flow_from_directory(TEST_DIR, class_mode="binary",target_size=(150,150))

### Question 5

Let's train our model for 10 more epochs using the same code as previously. Make sure you don't re-create the model - we want to continue training the model we already started training.

What is the mean of validation loss for all the epochs for the model trained with augmentations?

- 0.15
- 0.77
- 0.37
- 0.97

In [ ]:
history2=model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

In [ ]:
statistics.mean(history.history["val_loss"])

__ANSWER__: c) `0.37`

### Question 6
What's the average of validation accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

- 0.84
- 0.54
- 0.44
- 0.24

In [ ]:
statistics.mean(history.history["val_accuracy"][5:])

__ANSWER__: a) `0.84`